In [41]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from scipy.special import factorial

plt.rc('text', usetex=True)
plt.rc('font',**{'serif':['cm']})
plt.style.use('seaborn-paper')
plt.rcParams.update({'axes.titlesize':36})
plt.rcParams.update({'axes.labelsize': 30})
plt.rcParams.update({'xtick.labelsize': 26})
plt.rcParams.update({'ytick.labelsize': 26})
plt.rcParams.update({'legend.fontsize': 30})
plt.rcParams.update({'figure.titlesize': 36})



In [ ]:
markers=['o','s','x','d',(8,2,0)]
for label,func in {'European':Euro_GBM,'Asian':Asian_GBM,'Lookback':Lookback_GBM,'Digital':Digital_GBM}.items():
    option=func()
    complexity_plot(option,[0.005,0.01,0.02,0.05,0.1],markers=markers,label=label+'GBM')
    plt.savefig(label+'GBM_cost')
    variance_plot(option,0.005,label=label+'GBM')
    plt.savefig(label+'GBM_var')
    

In [ ]:
markers=['o','s','x','d',(8,2,0)]
for label,func in {'Asian':Asian_Merton,'Lookback':Lookback_Merton,'Digital':Digital_Merton}.items():
    option=func()
    complexity_plot(option,[0.01,0.02,0.05,0.07,0.1],markers=markers,label=label+'MJD')
    plt.savefig(label+'MJD_cost')
    variance_plot(option,0.005,label=label+'MJD')
    plt.savefig(label+'MJD_var')

In [ ]:
##Testing f(M) and actual cost
cost_mlmc=[]
cost_mc=[]
for m in [2,3,4,5,6,7,8,9,10]:
    Euro=Euro_GBM()
    e=0.005
    sums,N=Euro.mlmc(e,M=m)
    L=len(N)-1
    means_p=sums[2,:]/N
    V_p=(sums[3,:]/N)-means_p**2

    cost_mlmc+=[(N[0]+(1+1/m)*np.sum(N[1:]*(m**np.arange(1,L+1))))*e**2]
    cost_mc+=[2*sum(V_p*m**np.arange(L+1))]

M=np.array([2,3,4,5,6,7,8,9,10])
plt.plot(M,cost_mlmc,label='MLMC')
sc=cost_mlmc[3]/((M[3]-1/M[3])/(np.log(M[3]))**2)
plt.plot(M,sc*(M-1/M)/(np.log(M))**2,'r--',label='$f(M)$')
plt.legend()
plt.ylabel('$\epsilon^{2}$cost')
plt.xlabel('$M$');
plt.title(f'Asian Call, GBM, $\epsilon={e}$')
plt.savefig('Comparison_Cost_fofM')

In [ ]:
#MLMC,MC,BS Comparison
fig,ax=plt.subplots(2,2,figsize=(30,16),sharex=True)
eps=np.arange(0.01,0.11,0.01)
for label,func in {'European':Euro_GBM,'Asian':Asian_GBM,'Lookback':Lookback_GBM,'Digital':Digital_GBM}.items():
    option=func()
    p=[]
    if label=='European':
        axis=ax[0,0]
        axis.set_ylabel('Call Option Price')
    elif label=='Asian':
        axis=ax[0,1]
    elif label=='Lookback':
        axis=ax[1,0]
        axis.set_ylabel('Call Option Price')
        axis.set_xlabel('$\epsilon$')
    else:
        axis=ax[1,1]
        axis.set_xlabel('$\epsilon$')
    for e in eps:
        sums,N=option.mlmc(e,M=2)
        p+=[np.sum(sums[0,:]/N)]
    axis.plot(eps,p,'k--',label='MLMC')
    axis.fill_between(eps, p+eps, p-eps, alpha=.5,label='$\pm\epsilon$')
    axis.plot(eps,option.BS()*np.ones(len(eps)),'k-',label='Black-Scholes')
    axis.set_title(label)
lgd=plt.legend(loc="upper left", bbox_to_anchor=(1,1.1))
text=fig.suptitle(f'GBM Call Options, $T={option.T}, S(0)={option.X0}, K={option.K}, r={option.r*100}\%, \sigma={option.sig}$',fontsize=36);
fig.tight_layout(rect=[0, 0.03, 1, 0.94],w_pad=2,h_pad=2)
plt.savefig('Nice_Images/BSComparison_GBM',bbox_extra_artists=(lgd,text), bbox_inches='tight')

In [ ]:
##Testing Control Variate Factor
plt.rcParams['legend.title_fontsize'] = 24
plt.figure(figsize=(16,12))
e=0.005
M=2
plt.title(f'GBM SDE, $\lambda^*$\n$M={M}, \epsilon={e}$');
markers=['o','s','x','d']
i=0

for label,func in {'European':Euro_GBM,'Asian':Asian_GBM,'Lookback':Lookback_GBM,'Digital':Digital_GBM}.items():
    option=func()
    sums,N=option.mlmc(e,M=M)
    Vg=sums[5,:]/N-(sums[4,:]/N)**2
    lam=((sums[6,:]/N)-(sums[2,:]/N)*(sums[4,:]/N))/Vg
    plt.plot(range(len(N)),lam,'k-',marker=markers[i],label=label,
               markersize=20,markerfacecolor="None",markeredgecolor='k', markeredgewidth=1)
    i+=1

plt.xlabel('$l$')
plt.ylabel('$\lambda^{*}$')
plt.gca().xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
plt.legend(title='Call Option',fontsize=14,labelspacing=1)
plt.savefig('lambdaCallOptions_GBM')

In [ ]:
##Testing control variate factor
plt.rcParams['legend.title_fontsize'] = 24
plt.figure(figsize=(16,12))
M=2
e=0.05
markers=['o','s','x','d']
i=0
plt.title(f'Merton JD SDE, $\lambda^*$\n$M={M}, \epsilon={e}$');
plt.xlabel('$l$')
plt.ylabel('$\lambda^{*}$')

for label,func in {'European':Euro_Merton,'Asian':Asian_Merton,'Lookback':Lookback_Merton,'Digital':Digital_Merton}.items():
    option=func()
    sums,N=option.mlmc(e,M=M)
    Vg=sums[5,:]/N-(sums[4,:]/N)**2
    lam=((sums[6,:]/N)-(sums[2,:]/N)*(sums[4,:]/N))/Vg
    plt.plot(range(len(N)),lam,'k-',marker=markers[i],label=label,
               markersize=20,markerfacecolor="None",markeredgecolor='k', markeredgewidth=1)
    i+=1
plt.gca().xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
plt.legend(title='Call Option',fontsize=14,labelspacing=1)
plt.savefig('lambdaCallOptions_Merton')

In [ ]:
##Testing linear regression
Euro=Euro_GBM()
lr=LinearRegression()
M=[2,3,4,5,6,7,8,9,10]
alphs=[]
for i in range():
    alphs=[]
    for m in M:
        sums,N=Euro.mlmc(0.01,M=m)
        L=len(N)-1
        Yl=np.abs(sums[0,:])/N
        X=np.arange(1,L+1).reshape(-1,1)
        l = lr.fit(X = X, y = np.log(Yl[1:]))
        alpha = max(0.5,-l.coef_[0]/np.log(m))
        alphs+=[alpha]
    plt.plot(M,alphs,label=f'{i}')
plt.legend()